# Tutorial: Train a classification model on MNIST dataset

在这个 tutorial 中，我们学习使用 PyTorch 框架训练图像分类模型。我们使用简单的 LeNet 网络，在数字分类数据集 MNIST 上进行训练和测试。
代码改编自 PyTorch 官方例程：https://github.com/pytorch/examples/blob/main/mnist

## Step 1：检查环境并安装 PyTorch

In [1]:
# check NVCC version
!which python

# check GCC version
!gcc --version

# check python version
!which python
!python -V

'which' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'gcc' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'which' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


Python 3.8.12


In [2]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


## Step 2：使用 PyTorch 实现 LeNet 网络结构

LeNet 是 Yann LeCun 等人在 1998 年提出的一个经典的卷积神经网络，用于手写数字的识别，其网络结构如下：

![LeNet](lenet.png)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

# create LeNet class
class LeNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)  # C1: Nx6x28x28
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # S2: Nx6x14x14
        x = self.conv2(x)  # C3: Nx16x10x10
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  # S4: Nx16x5x5
        x = x.flatten(start_dim=1)  # Nx400
        x = self.fc1(x)  # F5: Nx120
        x = F.relu(x)
        x = self.fc2(x)  # F6: Nx84
        x = F.relu(x)

        output = self.fc3(x)  # Nx10
        return output
        
model = LeNet()  # 构造 LeNet 实例
model.cuda()  # 使用 GPU
print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Step 3: 使用 PyTorch 提供的 SGD 优化器

In [6]:
# torch.optim 提供了各种优化算法的实现
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

print(scheduler.get_last_lr())

[0.01]


## Step 4: 准备 MNIST 数据集

在 PyTorch 的辅助工具箱 Torchvision 中，提供了包括 MNIST 在内的多个常用数据集的接口，以及常用的数据预处理和曾广算法。

In [8]:
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms

# 使用 torchvision MNIST 数据接口构建 Dataset
transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

train_dataset = datasets.MNIST('./data',
                    train=True,
                    download=True,
                    transform=transform)

test_dataset = datasets.MNIST('./data',
                    train=False,
                    download=True,
                    transform=transform)


# 构建 DataLoader
train_loader = DataLoader(train_dataset,
                batch_size=64,
                shuffle=True,
                pin_memory=True)
test_loader = DataLoader(test_dataset,
                batch_size=1000,
                shuffle=False,
                pin_memory=True)

print('Length of train_dataset:', len(train_dataset))
print('Length of train_loader', len(train_loader))

print('Length of test_dataset:', len(test_dataset))
print('Length of test_loader', len(test_loader))

Length of train_dataset: 60000
Length of train_loader 938
Length of test_dataset: 10000
Length of test_loader 10


## Step 5: 训练和测试模型

In [9]:
import torch

# 训练一个 epoch 的过程
def train_epoch(model, loader, optimizer, epoch):
    # 将模型设为训练模式
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        # 从数据集中取出 1 个 batch
        data, target = data.cuda(), target.cuda()
        # 清除之前的梯度信息
        optimizer.zero_grad()
        # 前向传播 forward propagation
        output = model(data)
        # 计算损失函数
        loss = F.cross_entropy(output, target)
        # 反向传播 backward propagation
        loss.backward()
        # 优化器更新模型参数
        optimizer.step()

        # 显示训练信息
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tLR:{:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), optimizer.param_groups[0]['lr']))

# 测试模型的过程
def test(model, test_loader):
    # 将模型设为测试模式
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            # 统计损失函数值和正确分类样本数
            test_loss += F.cross_entropy(
                output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(
                dim=1,
                keepdim=True)  # get the index of the prediction
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    # 显示平均损失函数值和分类正确率信息
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

# 进行多轮训练，每轮训练后测试模型精度
total_epoch = 14

for epoch in range(total_epoch):
    train_epoch(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    # 更新学习率
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.318036	LR:0.010000
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.295588	LR:0.010000
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.288706	LR:0.010000
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.291859	LR:0.010000
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.298126	LR:0.010000
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.305079	LR:0.010000
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.290139	LR:0.010000
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.293179	LR:0.010000
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.287431	LR:0.010000
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.277005	LR:0.010000
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.247815	LR:0.010000
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.270228	LR:0.010000
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.250198	LR:0.010000
Train Epoch: 0 [8320/60000 (14%)]	Loss: 2.264555	LR:0.010000
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.253788	LR:0.010000
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.230576	LR:0.010000
Train Epoch: 0 [10240/60000 (17%)]	Lo

In [10]:
torch.save(model.state_dict(), './lenet_mnist.pth')

## Step 6: 模型加载和推理

当完成训练后，就可以加载训练好的模型参数，并用训练好的模型来推理（即对数据进行分类）。

In [11]:
import torch

# 加载训练好的模型
model = LeNet()
model.load_state_dict(torch.load('lenet_mnist.pth'))
model.cuda()

print(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [20]:
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

# 读取图像并显示
img = Image.open('image.png')
display(img)

# 使用和训练时相同的预处理（数据曾广除外）
transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])
data = transform(img).view(1, 1, 28, 28).cuda()

# 将数据送入模型，得到推理结果
pred = model(data)
pred = F.softmax(pred, dim=1)  # 将输出转化为概率
for label, prob in enumerate(pred[0].cpu().tolist()):
    print(f'Predicted probability of {label} is: {prob:06f}')



Predicted probability of 0 is: 0.000001
Predicted probability of 1 is: 0.000000
Predicted probability of 2 is: 0.000028
Predicted probability of 3 is: 0.000280
Predicted probability of 4 is: 0.000000
Predicted probability of 5 is: 0.000003
Predicted probability of 6 is: 0.000000
Predicted probability of 7 is: 0.999502
Predicted probability of 8 is: 0.000001
Predicted probability of 9 is: 0.000185
